<center> <h1> <span style="color:black"> Hands-on Machine Learning with Python  </h1> </center> 
<center> <h2> <span style="color:red"> Module 2: Tree-based machine learning methods </h1> </center>
<center> <h3> <span style="color:red"> Session 2: Ensembles </h1> </center>

# Structure of the notebook

* [Chapter 1 - Introduction](#one)
    + [1.1 Objectives of the notebook](#one-one)
    + [1.2 Library requirements](#one-two)

* [Chapter 2 - Bagging](#two)
    + [2.1 DIY example](#two-one)
    + [2.2 Regression](#two-two)
    + [2.3 Out-of-bag](#two-three)
    + [2.4 Grid search CV](#two-four)
        
* [Chapter 3 - Random forest](#three)
    + [3.1 Dominant features](#three-one)
    + [3.2 Feature sampling](#three-two)
    + [3.3 Tuning & insights](#three-three)

* [Chapter 4 - Boosting](#four)
    + [4.1 Parameters](#four-one)
    + [4.2 Out-of-bag](#four-two)
    + [4.3 Early stopping](#four-three)
    + [4.4 Other implementations](#four-four)

* [Chapter 5 - XGBoost](#five)
    + [5.1 Claim frequency](#five-one)
    + [5.2 Claim severity](#five-two)
    + [5.3 Random search CV](#five-three)


# Chapter 1 - Introduction <a name="one"></a>

## 1.1 Objectives of the notebook <a name="one-one"></a>
The objectives of this notebook are to:
1. Build tree-based ensembles for typical regression, classification and actuarial problems.
1. Tune the parameters of tree-based ensembles to obtain optimal performance.
1. Inspect ensembles to gain insights in the underlying decision process.

## 1.2 Library requirements <a name="one-two"></a>
We start by importing all the required Python packages for this notebook.

In [ ]:
# import packages
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotnine import ggplot, geom_point, geom_line, geom_vline, geom_hline, aes, theme_set, theme_bw
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.utils import shuffle
from sklearn.inspection import permutation_importance, partial_dependence, PartialDependenceDisplay
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb

# set the black and white theme for ggplot to get rid of gray backgrounds
theme_set(theme_bw())

# Chapter 2 - Bagging <a name="two"></a>
Bagging stands for **b**ootstrap **agg**regat**ing** and is a technique that can be applied to different types of base learners.  We will focus on decision trees as base learners.

## 2.1 DIY example <a name="two-one"></a>
We start by showcasing a small DIY experiment where we perform the two steps in bagging ourself:
1. bootstrap: sample the data with replacement.
2. aggregating: combine the models/predictions.

The goal is to understand the principles of how bagging models are set up..

We simulate data from a sinusoidal pattern with some normally distributed noise on top of it:

In [ ]:
# set a seed for reproducibility
np.random.seed(5678)
# generate a x array from 0 to 2*pi
x = np.linspace(start=0, stop=2*math.pi, num=500)
# generate the true model m as the sin of x
m = np.sin(x)
# generate the observed y by adding normal noise to m
y = m + np.random.normal(loc=0, scale=0.5, size = len(m))
# collect the arrays in a dataframe
dfr = pd.DataFrame.from_dict({'x':x, 'm':m, 'y':y})
# print the dataframe
dfr

The simulated data (gray points) and the underlying true model (green line) look as follows:

In [ ]:
# plot simulated data
ggplot(dfr, aes(x = 'x')) + geom_point(aes(y = 'y'), alpha = 0.3) + geom_line(aes(y = 'm'), colour = 'darkgreen', size = 1.5)

We will create our first bootstrap sample by taking observations at random with replacement via the `random.choices` function:

In [ ]:
# check out the difference between sample and choices
random.seed(12345)
print(random.choices(range(5),k=5))
print(random.sample(range(5),k=5))

In [ ]:
# get the number of rows
nrow_dfr = dfr.shape[0]
# set a seed for reproducibility
random.seed(1234)
# get a bootstrap sample with replacement
id1 = random.choices(range(nrow_dfr), k=int(0.8*nrow_dfr))
# subset the observations
dfr_b1 = dfr.iloc[id1]
# show the dataframe
dfr_b1.sort_values('x')

We do the exact same steps again, but now with a different seed to obtain a different sample:

In [ ]:
# set a seed for reproducibility
random.seed(5678)
# get a bootstrap sample with replacement
id2 = random.choices(range(nrow_dfr), k=int(0.8*nrow_dfr))
# subset the observations
dfr_b2 = dfr.iloc[id2]
# show the dataframe
dfr_b2.sort_values('x')

After creating our bootstrap datasets, we now fit a decision tree to each sample of the data:

In [ ]:
# fit a deep DecisionTreeRegressor to dfr_b1
tree_bag1 = DecisionTreeRegressor(criterion='squared_error', max_depth=5, min_samples_leaf=3).fit(dfr_b1.x.to_numpy().reshape(-1, 1), dfr_b1.y.to_numpy())
# fit a deep DecisionTreeRegressor to dfr_b2
tree_bag2 = DecisionTreeRegressor(criterion='squared_error', max_depth=5, min_samples_leaf=3).fit(dfr_b2.x.to_numpy().reshape(-1, 1), dfr_b2.y.to_numpy())

After fitting each tree, we now make predictions and aggregate them by taking the average for each observation over both trees:

In [ ]:
# get original x values
preds_bag = pd.DataFrame({'x':dfr.x})
# predictions for first tree
preds_bag['pred_bag1'] = tree_bag1.predict(dfr.x.to_numpy().reshape(-1,1))
# predictions for second tree
preds_bag['pred_bag2'] = tree_bag2.predict(dfr.x.to_numpy().reshape(-1,1))
# average both predictions 
preds_bag['pred_mean'] = (preds_bag['pred_bag1'] + preds_bag['pred_bag2']) / 2
# inspect the dataframe
preds_bag

For plotting purposes we transform the data from a wide to a long format via the `pandas.melt` function:

In [ ]:
# melt to long format
preds_bag_long = pd.melt(preds_bag, id_vars=['x'], var_name='model', value_name='pred')

We now plot the predictions for both individual trees and the aggregated prediction from our bagged model:

In [ ]:
# plot predictions
ggplot(preds_bag_long, aes(x = 'x')) + geom_line(aes(y = 'pred', colour = 'model'), size = 1, alpha = 0.5)

We can observe that the averaged prediction flattens some of the extreme predictions by one of the individual trees. Let's take this one step further.

**Your turn!**

* Create a third bootstrap sample and fit a decision tree to it. Be sure to take a different seed!
* Average the predictions over three trees and inspect the result.

In [ ]:
# add your code here

## 2.2 Regression <a name="two-two"></a>
A `scikit-learn` bagging model for regression is implemented in the `sklearn.ensemble.BaggingRegressor` class: [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html#sklearn.ensemble.BaggingRegressor).

*class sklearn.ensemble.BaggingRegressor(estimator=None, n_estimators=10, *, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0, base_estimator='deprecated')*

Let's fit a bagged tree ensemble for our toy regression dataset:

In [ ]:
# get X and y data
X = dfr.x.to_numpy().reshape(-1,1)
y = dfr.y.to_numpy()

In [ ]:
# initialize the bagging model
bag_reg = BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=25, min_samples_leaf=3), n_estimators=100, max_samples=0.8, bootstrap=True, oob_score=True, random_state=0)
# fit the bagging model
bag_reg.fit(X,y)

We can now visualize the result from our bagged model:

In [ ]:
# function to plot data and predictions for the regression example
def plot_reg(dfr, pred=None):
  dfr['pred'] = pred
  ggout = ggplot(dfr, aes(x = 'x')) + geom_point(aes(y = 'y'), alpha = 0.3) + geom_line(aes(y = 'm'), colour = 'darkgreen', size = 1.5)
  if pred is not None:
    ggout = ggout + geom_line(aes(y = 'pred'), colour = 'darkred', size = 1.5)
  return(ggout)

In [ ]:
# plot prediction result
plot_reg(dfr, pred = bag_reg.predict(X))

**Your turn!**

* Experiment with the parameter settings of the bagging ensemble and base learner to obtain a smooth fit.

In [ ]:
# add your code here

## 2.3 Out-of-bag <a name="two-three"></a>
Models where we take bootsrap samples come with a nice feature, namely the **o**ut-**o**f-**b**ag (OOB) prediction and error: 
1. The OOB prediction for an observation is calculated by taking only those models where this instance was not represented in the bootstrap training sample, and averaging these predictions. 
1. The OOB error is then calculated by comparing the OOB prediction for each observation to the true value and this serves as a generalization measure like a cross-validation error. The big advantage of OOB is that it comes for free without the need for extra model fits, like for example in 5-fold CV. 

Both the predictions and score are returned as an attribute of a fitted model if `oob_score = True`:

In [ ]:
# OOB predictions
bag_reg.oob_prediction_

In [ ]:
# OOB score
bag_reg.oob_score_

Notice that this is the $R^2$ metric:

In [ ]:
# R2 score
from sklearn.metrics import r2_score
r2_score(y, bag_reg.oob_prediction_)

We can also calculate the OOB MSE in one of the following two ways:

In [ ]:
# MSE DIY
np.mean((bag_reg.oob_prediction_ - y)**2)

In [ ]:
# MSE sklearn
from sklearn.metrics import mean_squared_error
mean_squared_error(y, bag_reg.oob_prediction_)

We evaluate the OOB MSE for different number of estimators (trees) in the ensemble:

In [ ]:
# initialize a vector for the number of estimators and oob score
num_est = [5,10,20,50,75,100,200,400,800]
oob_mse = np.zeros(len(num_est))
# iterate over the list
for i in range(len(num_est)):
  print(num_est[i])
  # fit a bagged model
  bag_reg = BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=25, min_samples_leaf=3), n_estimators=num_est[i], max_samples=0.8, bootstrap=True, oob_score=True, random_state=0).fit(X,y)
  # evaluate the OOB MSE
  oob_mse[i] = mean_squared_error(y, bag_reg.oob_prediction_)
# Collect ans inspect the results
pd_oob = pd.DataFrame.from_dict({'num_est':num_est,'oob_mse':oob_mse})
pd_oob

Remember the nice thing about this OOB error: it is a generalization metric that comes for free during model fitting. Next we will see how you can perform grid search cross-validation on a bagged regression model.

## 2.4 Grid search CV <a name="two-four"></a>
The exercise from the Section 2.2 showed us that it is possible to obtain a good model fit by manually tweaking some parameters. However, there are two very big drawbacks with that approach:
1. This manual tweaking is time-consuming work and not fun to do.
1. Validation of good happened on a visual basis but not in a quantitative way.

A parameter grid search via cross-validation mediates both issues, giving an automatic way to try different settings and returning a quantifiable loss metric to base decisions on regarding what a "good" fit is. In `scikit-learn`, grid search CV is implemented in the `class sklearn.model_selection.GridSearchCV`: [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

We define a prameter grid and perform 5-fold CV to find the optimal values in the grid as follows:

In [ ]:
# shuffle the feature matrix and target vector
Xs, ys = shuffle(X,y, random_state= 0)
# define a parameter grid as a dict
param_grid = {
'estimator__max_depth' : [5, 10, 20],
'n_estimators' : [20, 50, 100]
}
# initialize the model
bag_reg = BaggingRegressor(estimator=DecisionTreeRegressor(min_samples_leaf=3), max_samples=0.8, bootstrap=True, oob_score=True, random_state=0)
# initialize the 5-fold CV
bag_reg_cv = GridSearchCV(bag_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
# fit the CV
bag_reg_cv.fit(Xs,ys)

We can collect the results from the `cv_results_` attribute:

In [ ]:
# inspect results
bag_reg_cv.cv_results_

In [ ]:
# collect results
results_cv = bag_reg_cv.cv_results_
# store in a dataframe
results_pd = pd.DataFrame.from_dict(
    {'depth':results_cv['param_estimator__max_depth'].data,
     'estimators':results_cv['param_n_estimators'].data,
     'score':-results_cv['mean_test_score'],
     'rank':results_cv['rank_test_score']}).sort_values('rank')
# show the top results
results_pd.iloc[0:6]

We now plot the predictions for the optimal values according to our grid search:

In [ ]:
# obtain the optimal alpha from the CV results
opt_depth = results_pd[results_pd['rank'] == 1]['depth'].iloc[0]
opt_estim = results_pd[results_pd['rank'] == 1]['estimators'].iloc[0]
# calculate the predictions for this alpha value
pred = BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=opt_depth, min_samples_leaf=3), n_estimators=opt_estim, max_samples=0.8, bootstrap=True, oob_score=True, random_state=0).fit(X,y).predict(X)
# plot the predictions
plot_reg(dfr,pred)

**Your turn!**

* Expand the current grid search for the regression example to find an even better fit.
* Experiment with the classification toy example of session 1 to fit a bagged classification model.

In [ ]:
# data generation classification example
np.random.seed(54321)
x1 = np.repeat(np.arange(0.1, 10.1, 0.1), 100)
x2 = np.tile(np.arange(0.1, 10.1, 0.1), 100)
X_clf = np.stack([x1,x2], axis=1)
y_clf = np.zeros(len(x1), dtype=int)
y_clf += (x1 + 2*x2 < 8).astype(int)
y_clf += (3*x1 + x2 > 30).astype(int)
y_clf += np.round(np.random.normal(loc=0, scale=0.3, size=len(y_clf))).astype(int)
y_clf = np.clip(y_clf, 0, 1)
dfc = pd.DataFrame.from_dict({'x1':x1,'x2':x2,'y':y_clf})
dfc['y'] = dfc['y'].astype("category")

In [ ]:
# add your code here

# Chapter 3 - Random forest <a name="three"></a>
A `scikit-learn` random forest regressor is implemented in the `sklearn.ensemble.RandomForestRegressor`: [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor).

*class sklearn.ensemble.RandomForestRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=1.0, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)*

## 3.1 Dominant features <a name="three-one"></a>
We start the introduction of random forests by showing the need for random feature sampling in the case of dominant features within the MTPL dataset. Let's read and pre-process the data:

In [ ]:
# read the MTPL data
mtpl = pd.read_csv("https://katrienantonio.github.io/hands-on-machine-learning-R-module-1/data/PC_data.txt", delimiter = "\t", usecols=list(range(1,14)))
# transform the column names to lowercase
mtpl.columns = mtpl.columns.str.lower()
# rename the exp column to expo
mtpl = mtpl.rename(columns= {'exp': 'expo'})
# map string values to integers for certain columns
mtpl['coverage'] = mtpl['coverage'].map({'TPL':0, 'PO':1, 'FO':2})
mtpl['fleet'] = mtpl['fleet'].map({'N':0, 'Y':1})
mtpl['fuel'] = mtpl['fuel'].map({'gasoline':0, 'diesel':1})
mtpl['use'] = mtpl['use'].map({'private':0, 'work':1})
mtpl['sex'] = mtpl['sex'].map({'male':0, 'female':1})
# print the shape
print(mtpl.shape)
# show the first observations
mtpl.head(100)

We create data objects for the features `X`, the target `y` and the weights `w`:

In [ ]:
# cols to retain as features
feat_cols = ['bm','ageph','agec','power','coverage','fuel','sex','fleet','use']
# subset the data
X_mtpl_freq = mtpl[feat_cols]
# print the shape
print(X_mtpl_freq.shape)
# show the features
X_mtpl_freq

# claim frequency (nclaims/expo) as target
y_mtpl_freq = np.array(mtpl.nclaims/mtpl.expo)
# exposure as weights
w_mtpl_freq = np.array(mtpl.expo)

We create two arrays which contain random bootstrap sample ids for the MTPL data:

In [ ]:
nrow_mtpl = mtpl.shape[0]
# get a bootstrap sample with replacement
random.seed(1234)
id1 = random.choices(range(nrow_mtpl), k=int(0.8*nrow_mtpl))
# get a bootstrap sample with replacement
random.seed(5678)
id2 = random.choices(range(nrow_mtpl), k=int(0.8*nrow_mtpl))

Next we fit a decision tree to each subset of the MTPL data:

In [ ]:
# initialize and fit the tree to bootstrap sample 1
tree_freq1 = DecisionTreeRegressor(criterion='poisson', max_depth=2, min_samples_split=10000, min_samples_leaf=5000)
tree_freq1.fit(X=X_mtpl_freq.iloc[id1], y=y_mtpl_freq[id1], sample_weight=w_mtpl_freq[id1])
# plot the tree
plt.figure(figsize=(6, 5), dpi=100)
plot_tree(tree_freq1, feature_names=X_mtpl_freq.columns);

In [ ]:
# initialize and fit the tree to bootstrap sample 2
tree_freq2 = DecisionTreeRegressor(criterion='poisson', max_depth=2, min_samples_split=10000, min_samples_leaf=5000)
tree_freq2.fit(X=X_mtpl_freq.iloc[id2], y=y_mtpl_freq[id2], sample_weight=w_mtpl_freq[id2])
# plot the tree
plt.figure(figsize=(6, 5), dpi=100)
plot_tree(tree_freq2, feature_names=X_mtpl_freq.columns);

Notice how similar both trees are due to the dominance of the bonus malus feature? Aggregating similar trees will not result in a great reduction of variance and will therefore limit the predictive performance of the ensemble.

## 3.2 Feature sampling <a name="three-two"></a>
We fit a random forest regressor to the MTPL data and randomly sample features via the `max_features` parameter:

In [ ]:
# initialize a random forest
rf_freq = RandomForestRegressor(n_estimators=100, max_features = 0.3, criterion='poisson', max_depth=2, min_samples_split=10000, min_samples_leaf=5000, random_state=0)
# fit the tree to our target with weights
rf_freq.fit(X=X_mtpl_freq, y=y_mtpl_freq, sample_weight=w_mtpl_freq)
# print the tree
rf_freq

Let's take two random trees from the ensemble and inspect the first splits and which features are chosen:

In [ ]:
# plot a tree from the ensemble
plt.figure(figsize=(6, 5), dpi=100)
plot_tree(rf_freq.estimators_[20], feature_names=X_mtpl_freq.columns);

In [ ]:
# plot a tree from the ensemble
plt.figure(figsize=(6, 5), dpi=100)
plot_tree(rf_freq.estimators_[90], feature_names=X_mtpl_freq.columns);

Due to the random feature sampling at each split we now have individual trees that are more decorrelated to improve variance reduction, hence a random forest.

## 3.3 Tuning & insights <a name="three-three"></a>
We now perform a simple grid search to find a good claim frequency random forest and use interpretation tools to gain insights from our model.

In [ ]:
# define a parameter grid as a dict
param_grid = {
'n_estimators' : [50, 100],
'max_features' : [0.6, 1],
'max_depth' : [5, 10]
}
# initialize the model and CV
rf_freq = RandomForestRegressor(criterion='poisson', min_samples_split=10000, min_samples_leaf=2000, random_state=0)
rf_freq_cv = GridSearchCV(rf_freq, param_grid, cv=5, scoring='neg_mean_poisson_deviance')
# fit the CV
rf_freq_cv.fit(X_mtpl_freq, y=y_mtpl_freq, sample_weight=w_mtpl_freq)

Check the CV results:

In [ ]:
# collect results
results_cv = rf_freq_cv.cv_results_
# store in a dataframe
results_pd = pd.DataFrame.from_dict(
    {'depth':results_cv['param_max_depth'].data,
     'estimators':results_cv['param_n_estimators'].data,
     'features' : results_cv['param_max_features'].data,
     'score':-results_cv['mean_test_score'],
     'rank':results_cv['rank_test_score']}).sort_values('rank')
# show the top results
results_pd.iloc[0:6]

Extract the best estimator from the CV results:

In [ ]:
# get the best model from the CV results
rf_freq_opt = rf_freq_cv.best_estimator_

Show the feature importance metric:

In [ ]:
# collect the feature names and importance scores
rf_freq_fi = pd.DataFrame({'feature':rf_freq_opt.feature_names_in_, 'importance':rf_freq_opt.feature_importances_}).sort_values('importance', ascending=False)
# inspect the results
rf_freq_fi

Show some the PDP for all features in the MTPL data:

In [ ]:
# create pdps for a couple of features
fig, ax = plt.subplots(figsize=(15, 10))
PartialDependenceDisplay.from_estimator(rf_freq_opt, X_mtpl_freq, features = ['bm','ageph','power','fuel','agec','coverage'], categorical_features=['fuel','coverage'], kind='average', ax=ax);

What do you think of these effects compared to those from the decision trees?

**Your turn!**

* Experiment with the grid search to find a better performing model. Beware of the tuning time needed.
* How do those PDPs look?

In [ ]:
# add your code here

# Chapter 4 - Boosting <a name="four"></a>
A `scikit-learn` gradient boosting regressor is implemented in the `sklearn.ensemble.GradientBoostingRegressor`: [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor).

*class sklearn.ensemble.GradientBoostingRegressor(*, loss='squared_error', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)*

## 4.1 Parameters <a name="four-one"></a>
Let's inspect how certain parameters affect the boosting process.

In [ ]:
# initialize the boosting model
bst_reg = GradientBoostingRegressor(loss='squared_error', learning_rate=1, n_estimators=5, max_depth=1, min_samples_leaf=5, subsample=0.8, random_state=0)
# fit the boosting model
bst_reg.fit(X,y)
# plot prediction result
plot_reg(dfr, pred = bst_reg.predict(X))

What if we lower the learning rate?

In [ ]:
# initialize the boosting model
bst_reg = GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=5, max_depth=1, min_samples_leaf=5, subsample=0.8, random_state=0)
# fit the boosting model
bst_reg.fit(X,y)
# plot prediction result
plot_reg(dfr, pred = bst_reg.predict(X))

What if we increase the individual tree depth?

In [ ]:
# initialize the boosting model
bst_reg = GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=5, max_depth=2, min_samples_leaf=5, subsample=0.8, random_state=0)
# fit the boosting model
bst_reg.fit(X,y)
# plot prediction result
plot_reg(dfr, pred = bst_reg.predict(X))

What if we increase the number of boosting iterations?

In [ ]:
# initialize the boosting model
bst_reg = GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=50, max_depth=2, min_samples_leaf=5, subsample=0.8, random_state=0)
# fit the boosting model
bst_reg.fit(X,y)
# plot prediction result
plot_reg(dfr, pred = bst_reg.predict(X))

Let's visualize the iteration process over time:

In [ ]:
# initialize a plot object
plt.figure(figsize=(20, 20),dpi=100)
plt.subplots_adjust(hspace=0.5)
plt.suptitle("Boosting predictions for different iterations", fontsize=18, y=0.95)
# iterate over the tree list and take every 5th item
for i, indx in enumerate(list(range(1, 90, 10))): 
  # fit and make a prediction for this iteration
  pred = GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=indx, max_depth=2, min_samples_leaf=5, subsample=0.8, random_state=0).fit(X,y).predict(X)
  # plot the predictions in a subplot
  ax = plt.subplot(3, 3, i + 1)
  ax.scatter(x,y, color='gray', s=2)
  ax.plot(x,m,color='green')
  ax.plot(x,pred,color='red')
  ax.set_title(f'iteration: {indx}')
  ax.set_xlabel('x')
  ax.set_ylabel('y')

**Your turn!**

Adjust the `learning_rate` parameter in the cell above to the following values and explain what you see:
- `learning_rate` = 1
- `learning_rate` = 0.01

Is this what you would expect?

## 4.2 Out-of-bag <a name="four-two"></a>
By setting `subsample` < 1, we are performing stochastic boosting where each individual tree base learner is fit on a sample of the original dataset. This means that, just like with bagging, we have observations that are not being used by certain base learners are are therefore out-of-bag.

Let's retake the boosting regressor from before but with more boosting iterations:

In [ ]:
# initialize the boosting model
bst_reg = GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=100, max_depth=2, min_samples_leaf=5, subsample=0.8, random_state=0)
# fit the boosting model
bst_reg.fit(X,y)

The improvement in the OOB error per boosting iteration is saved as the `ob_improvement_` attribute of a fitted boosting model: 

In [ ]:
# get oob improvement
bst_reg.oob_improvement_

We can visualize the OOB error improvement over time as follows:

In [ ]:
# save the oob improvement in a dataframe
bst_oob = pd.DataFrame.from_dict({'iteration':range(1,bst_reg.n_estimators_+1),'oob_improv':bst_reg.oob_improvement_})
# plot the evolution of OOB improvement over time
ggplot(bst_oob, aes(x='iteration', y='oob_improv')) + geom_line()

Notice how the improvement becomes negligible and even negative (indicating worse performance) after a certain number of iterations. This can point towards overfitting as the OOB error serves as a generalization error metric. The number of boosting iterations is of course a tuning parameter, but we can also make use of another interesting trick.

## 4.3 Early stopping <a name="four-three"></a>
Early stopping is a technique to stop boosting iterations before the requested number of trees `n_estimators` is reached. This is done by tracking performance over time for a specific validation set and can be implemented by setting the following parameters:
- `n_iter_no_change`: early stop if the loss does not improve for this many iterations (default `None` to disable early stopping).
- `validation_fraction`: proportion of training data to use for early stopping loss (default = 0.1).
- `tol`: tolerance for the early stopping loss (default = 1e-4). 

When the loss on the validation set is not improving by at least `tol` for `n_iter_no_change` iterations, the training stops.

Let's retake the previous example again, but now with early stopping enabled:

In [ ]:
# initialize the boosting model
bst_reg = GradientBoostingRegressor(loss='squared_error', learning_rate=0.1, n_estimators=100, max_depth=2, min_samples_leaf=5, subsample=0.8, random_state=0, n_iter_no_change = 5)
# fit the boosting model
bst_reg.fit(X,y)

We requested 100 boosting iterations, but we ended up with only:

In [ ]:
# number of estimators
bst_reg.n_estimators_

We can add indicators for the early stopping round and the tolerance to the evolution of the OOB error improvement:

In [ ]:
# plot early stop together with OOB
ggplot(bst_oob.query('iteration > 10'), aes(x='iteration', y='oob_improv')) + geom_line() + geom_vline(xintercept=bst_reg.n_estimators_) + geom_hline(yintercept=1e-4)

Early stopping is calculated on a validation set and the OOB error improvement on the out-of-bag samples, but both can lead to very similar conclusions about the optimal number of boosting iterations.

## 4.4 Other implementations <a name="four-four"></a>
`GradientBoostingRegressor`, the standard boosting implementation of `sklearn`, has three big disadvantages:
- slow for large datasets: no parallel computing and exhaustive algorithms
- limited number of options: no regularization and support for missing values for example
- limited number of loss functions: squared_error, absolute_error, huber, quantile

There are plenty of other boosting implementations available for more advanced modeling:
- `sklearn.ensemble.HistGradientBoostingRegressor`: [docs](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html#sklearn.ensemble.HistGradientBoostingRegressor)
- `lightgbm.LGBMRegressor`: [docs](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html#lightgbm.LGBMRegressor)
- `catboost.CatBoostRegressor`: [docs](https://catboost.ai/en/docs/concepts/python-reference_catboostregressor)
- `xgboost.XGBRegressor`: [docs](https://xgboost.readthedocs.io/en/stable/python/python_api.html#module-xgboost.sklearn)

We will be going into more detail about `xgboost` via the `scikit-learn` wrapper interface for XGBoost.

# Chapter 5 - XGBoost <a name="five"></a>
One of the most popular packages in python (and beyond) for stochastic gradient boosting machines is `xgboost`. This library provides an efficient implementation of the gradient boosting algorithm with some useful extra features.

The function `class xgboost.XGBRegressor(*, objective='reg:squarederror', **kwargs)` presents, among others, the following parameters:
* `n_estimators`: number of gradient boosted trees; this is equivalent to the number of boosting rounds
* `max_depth`: max. tree depth for base learners
* `learning_rate`: boosting learning rate
* `objective`: objective function to be used
* `tree_method`: which tree method to use, default = `auto`
* `reg_alpha`: L1 regularization term on weights
* `reg_lambda`: L2 regularization term on weights
* `monotone_constraints`: Constraint of variable monotonicity
* ...

A list of all available parameters can be found [here](https://xgboost.readthedocs.io/en/stable/parameter.html).

Let's build claim frequency and severity models for our MTPL dataset with XGBoost.

In [ ]:
# print mtpl data
mtpl

## 5.1 Claim frequency <a name="five-one"></a>
We first calculate the required targets and weights for our claim frequency regression problem like before:

In [ ]:
# cols to retain as features
feat_cols = ['bm','ageph','agec','power','coverage','fuel','sex','fleet','use']
# subset the data
X_mtpl_freq = mtpl[feat_cols]
# print the shape
print(X_mtpl_freq.shape)

# claim frequency (nclaims/expo) as target
y_mtpl_freq = np.array(mtpl.nclaims/mtpl.expo)
# exposure as weights
w_mtpl_freq = np.array(mtpl.expo)

Now we can fit an XGBoost model for claim frequency via the `count:poisson` objective:


In [ ]:
# initialize the xgboost model
xgb_freq = xgb.XGBRegressor(n_estimators = 500,
                            objective='count:poisson',
                            monotone_constraints = (1,0,0,0,0,0,0,0,0),
                            max_depth = 3,
                            learning_rate = 0.01,
                            base_score = np.sum(y_mtpl_freq * w_mtpl_freq)/np.sum(w_mtpl_freq))
# fit the xgboost model
xgb_freq.fit(X_mtpl_freq, y_mtpl_freq, sample_weight=w_mtpl_freq)

We can make predictions from this model on the target scale via the `predict` method:


In [ ]:
# prediction on the target scale
pred_on_target_scale = np.round(xgb_freq.predict(X_mtpl_freq), 5)
plt.hist(pred_on_target_scale, bins=10);

We obtain annual claim frequency predictions, without exposure being taken into account:

In [ ]:
# summary statistics of the predictions
pd.Series(pred_on_target_scale).describe()

We can make predictions on the log scale by setting `output_margin = True`:

In [ ]:
# prediction on raw log scale
pred_on_log_scale = np.round(xgb_freq.predict(X_mtpl_freq, output_margin = True), 5)
plt.hist(pred_on_log_scale, bins=10);

We can make predictions from a subset of the ensemble via the `iteration_range` parameter:

In [ ]:
# prediction on subset
pred_on_subset = np.round(xgb_freq.predict(X_mtpl_freq, iteration_range = (0,5)), 5)
plt.hist(pred_on_subset, bins=10);

## 5.2 Claim severity <a name="five-two"></a>
We first calculate the required targets and weights for our claim severity regression problem like before:

In [ ]:
# subset the data
mtpl_sev = mtpl.query('amount > 1 & amount < 100000')
X_mtpl_sev = mtpl_sev[feat_cols]
# print the shape
print(X_mtpl_sev.shape)

# claim severity (amount/nclaims) as target
y_mtpl_sev = np.array(mtpl_sev.avg)
# number of claims as weights
w_mtpl_sev = np.array(mtpl_sev.nclaims)

Now we can fit an XGBoost model for claim frequency via the `reg:gamma` objective:


In [ ]:
# initialize the xgboost model
xgb_sev = xgb.XGBRegressor(n_estimators = 100,
                           objective='reg:gamma',
                           max_depth = 3,
                           learning_rate = 0.01,
                           base_score = np.sum(y_mtpl_sev * w_mtpl_sev)/np.sum(w_mtpl_sev))
# fit the xgboost model
xgb_sev.fit(X_mtpl_sev, y_mtpl_sev, sample_weight=w_mtpl_sev)

We can make predictions from this model on the target scale via the `predict` method:

In [ ]:
# prediction on the target scale
pred_on_target_scale = np.round(xgb_sev.predict(X_mtpl_freq), 2)
plt.hist(pred_on_target_scale, bins=10);

We can observe that we are no longer underestimating the mean of our severity distribution like we were doing with the decision trees on the log-transformed severity:

In [ ]:
# mean of the predictions
pred_on_target_scale.mean()

In [ ]:
# mean of the portfolio severity
mtpl_sev.avg.mean()

## 5.3 Random search CV <a name="five-three"></a>
A grid search has the advantage that all possible combinations of tuning parameters are considered and the optimal combination is found. This procedure however becomes extremely time-consuming if a lot of tuning parameters are involved. In such a situation, a randomized search is better to save computation time. A randomized search simply tries *m* possible combinations out of *n* cases and returns the best performing one from this subset.

We start by defining our possible parameter values and initialize an `XGBRegressor` for claim frequency:

In [ ]:
# define dictionary for search
param_dict = {'max_depth' : [1, 3, 5, 7, 9],
              'n_estimators' : [100, 200, 300],
              'colsample_bynode' : [0.5, 0.75, 1],
              'lambda' : [0, 0.1, 1],
              'alpha' : [0, 0.1, 1]}

# Initialize an XGBRegressor
xgb_init = xgb.XGBRegressor(booster='gbtree',
                            learning_rate = 0.01,
                            objective='count:poisson',
                            eval_metric = 'poisson-nloglik',
                            monotone_constraints = (1,0,0,0,0,0,0,0,0),
                            base_score = np.sum(y_mtpl_freq * w_mtpl_freq)/np.sum(w_mtpl_freq))

The function `sklearn.model_selection.RandomizedSearchCV` presents the following additional (or other) parameters:

* `param_distributions`: the dictionary object that holds the distribution of possible hyperparameters you want to try
* `n_iter`: the number of random tuning parameter combinations to try out
* `random_state`: an initializer to make the random selection process repeatable.

In [ ]:
# perform cross_validation
xgb_randomsearch = RandomizedSearchCV(estimator=xgb_init,
                                      param_distributions=param_dict,
                                      scoring='neg_mean_poisson_deviance',
                                      n_iter=2,
                                      verbose=1,
                                      cv=5,
                                      random_state = 54321)
xgb_randomsearch.fit(X_mtpl_freq, y_mtpl_freq, sample_weight=w_mtpl_freq)

We can collect the random search results from the `cv_results_` attribute:

In [ ]:
# get cv results
pd.DataFrame(xgb_randomsearch.cv_results_).sort_values('rank_test_score')

We can get the best parameter combination from the `best_params_` attribute:

In [ ]:
# get best combination
xgb_randomsearch.best_params_

We can get the best model from the `best_estimator_` attribute:

In [ ]:
# get best model
xgb_best = xgb_randomsearch.best_estimator_
xgb_best

We can get the feature importance scores from the `feature_importances_` attribute:

In [ ]:
# get feature importance
pd.DataFrame({'feature':xgb_best.feature_names_in_, 'importance':xgb_best.feature_importances_}).sort_values('importance', ascending=False)

We can visualize some PDP effects via the `PartialDependenceDisplay.from_estimator` function:

In [ ]:
# visualize pdps
fig, ax = plt.subplots(figsize=(20, 6))
ax.set_title("PDPs")
PartialDependenceDisplay.from_estimator(xgb_best, X=X_mtpl_freq[0:1000], features = ['bm','ageph',('ageph','power')], ax=ax);

**Your turn!**

Time to put everything together and find the best possible claim frequency model:

* Choose 3 model classes, for example: a regression tree, a random forest and an xgboost model.
* Split the MTPL data in train and test data.
* Find the optimal tuning parameters for each model class by performing x-fold cross-validation via a grid or random search on the train data.
* Test the performance of each model on the out-of-sample test data.
* Which one is the winner?

In [ ]:
#add your code here